In [1]:
def flatten(outer_list):
    return [el for inner_list in outer_list for el in inner_list]
        

## 第5章: 係り受け解析  
夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．

In [2]:
path="./Section_5/neko.txt.cabocha"

In [3]:
!cat NLP/Section_5/neko.txt | cabocha -f1 > NLP/Section_5/neko.txt.cabocha

/bin/sh: NLP/Section_5/neko.txt.cabocha: No such file or directory
cat: NLP/Section_5/neko.txt: No such file or directory


In [4]:
!head -n 15 $path

* 0 -1D 0/0 0.000000
一	名詞,数,*,*,*,*,一,イチ,イチ
EOS
EOS
* 0 2D 0/0 -0.764522
　	記号,空白,*,*,*,*,　,　,　
* 1 2D 0/1 -0.764522
吾輩	名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 2 -1D 0/2 0.000000
猫	名詞,一般,*,*,*,*,猫,ネコ,ネコ
で	助動詞,*,*,*,特殊・ダ,連用形,だ,デ,デ
ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル
。	記号,句点,*,*,*,*,。,。,。
EOS


40. 係り受け解析結果の読み込み（形態素）  
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

In [5]:
class Morph:
    def __init__(self, dc):
        self.surface = dc['surface']
        self.base = dc['base']
        self.pos = dc['pos']
        self.pos1 = dc['pos1']

In [6]:
# 形態素Morphのリスト（一文）
def parseCabocha(block):
    res = []
    for line in block.split('\n'):
        if line == '':
            return res
        elif line[0] == '*':
            continue
        (surface, attr) = line.split('\t')
        attr = attr.split(',')
        lineDict = {
            'surface': surface,
            'base': attr[6],
            'pos': attr[0],
            'pos1': attr[1]
        }
        res.append(Morph(lineDict))

In [7]:
# blockList:文のリスト
with open(path, mode='rt', encoding='utf-8') as f:
    blockList = f.read().split('EOS\n')
    
# .split('EOS\n')により""要素がリストに含まれてしまうので除外
blockList = list(filter(lambda x: x != '', blockList))# [x for x in blockList if x != ""]

# parseCabocha(block) 形態素の辞書のリスト
blockList = [parseCabocha(block) for block in blockList]
for m in blockList[2]:
    print(vars(m))

{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'}
{'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


41. 係り受け解析結果の読み込み（文節・係り受け）  
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

In [8]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs    # 形態素（Morphオブジェクト）のリスト
        self.dst = dst          # 係り先文節インデックス番号
        self.srcs = []          # 係り元文節インデックス番号のリスト

In [9]:
def parseCabocha(block):
    def checkCreateChunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            res.append(c)
            tmp = []
        return tmp

    res = []
    tmp = []
    dst = None
    for line in block.split('\n'):# 文を行に分ける
        if line == '': 
            # これまでの形態素を文節クラスに格納
            tmp = checkCreateChunk(tmp)
        elif line[0] == '*':
            # 係り先文節インデックス番号 > dst
            # これまでの形態素を文節クラスに格納
            dst = line.split(' ')[2].rstrip('D')
            tmp = checkCreateChunk(tmp)
        else:
            # 文節中なら行を形態素クラスに格納し、保持
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            lineDict = {
                'surface': surface,
                'base': attr[6],
                'pos': attr[0],
                'pos1': attr[1]
            }
            tmp.append(Morph(lineDict))

    for i, r in enumerate(res):# 係り元index追加
        if int(r.dst)>=0:
            res[int(r.dst)].srcs.append(i)
    return res

In [10]:
with open(path, mode='rt', encoding='utf-8') as f:
    blockList = f.read().split('EOS\n')
blockList = list(filter(lambda x: x != '', blockList))

# parseCabocha(block) 文節のリスト
                    # res[i] 
blockList = [parseCabocha(block) for block in blockList]
for m in blockList[7]:
    print([mo.surface for mo in m.morphs], m.dst, m.srcs)

['この'] 7 []
['書生', 'という', 'の', 'は'] 4 []
['時々'] 4 []
['我々', 'を'] 5 []
['捕え', 'て'] 6 [1, 2]
['煮', 'て'] 7 [3]
['食う', 'という'] -1 [4]
['話', 'で', 'ある', '。'] -1 [0, 5]


42. 係り元と係り先の文節の表示  
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [11]:
for b in blockList[:5]:# b:parseCabocha(block)
    for m in b:# m:Chunk
        if int(m.dst) > -1:
            print(''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs]),
                  ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs]), sep='\t')
# 記号が係っていると判断されると空白に置き換わっているので注意

	猫である
名前は	無い
どこで	つかぬ
生れたか	つかぬ
とんと	つかぬ
何でも	所で
薄暗い	所で
じめじめした	泣いて
所で	泣いて
ニャーニャー	記憶している
泣いて	記憶している


43. 名詞を含む文節が動詞を含む文節に係るものを抽出  
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

In [12]:
for b in blockList[:5]:# b:parseCabocha(block)
    for m in b:# m:Chunk
        if int(m.dst) > -1:
            preText = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
            prePos = [mo.pos for mo in m.morphs]
            postText = ''.join([mo.surface if mo.pos != '記号' else '' for mo in b[int(m.dst)].morphs])
            postPos = [mo.pos for mo in b[int(m.dst)].morphs]
            # 「名詞を含む文節が，動詞を含む文節に係るとき」の条件を付与します。
            if '名詞' in prePos and '動詞' in postPos:
                print(preText, postText, sep='\t')

どこで	つかぬ
所で	泣いて
ニャーニャー	記憶している


44. 係り受け木の可視化  
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [13]:
# n = pydot.Node('node')
# n.fontname = "arialuni.ttf"
# n.fontsize = 9
# n.fontcolor = "blue"

In [14]:
import pydot
from graphviz import Digraph
graph_body = Digraph(format="png")
graph_body.attr('node', shape='box', fontname='MS Gothic')

In [15]:
pairs = []
target = blockList[7]
for m in target:# m:Chunk 
    if int(m.dst) > -1:
        preText = ''.join([mo.surface if mo.pos != '記号' else '' for mo in m.morphs])
        postText = ''.join([mo.surface if mo.pos != '記号' else '' for mo in target[int(m.dst)].morphs])
        pairs.append([preText, postText])

print(pairs)
g = pydot.graph_from_edges(pairs)

[['この', '話である'], ['書生というのは', '捕えて'], ['時々', '捕えて'], ['我々を', '煮て'], ['捕えて', '食うという'], ['煮て', '話である']]


In [16]:
# 日本語出力はできてない

45. 動詞の格パターンの抽出  
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．  
・動詞を含む文節において，最左の動詞の基本形を述語とする  
・述語に係る助詞を格とする  
・述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる  
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．  
>始める  で  
>見る    は を  

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語と格パターンの組み合わせ  
「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

In [17]:
# UNIXコマンドはパス

In [18]:
for b in blockList[:5]:
    for m in b:
        if len(m.srcs) > 0:
            preMorphs = [b[int(s)].morphs for s in m.srcs]
            preMorphs = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in preMorphs]
            preSurface = [[p.surface for p in pm] for pm in preMorphs]
            preSurface = list(filter(lambda x: x != [], preSurface))
            preSurface = [p[0] for p in preSurface]
            postBase = [mo.base for mo in m.morphs]
            postPos = [mo.pos for mo in m.morphs]
            if len(preSurface) > 0 and '動詞' in postPos:
                print(postBase[0], ' '.join(preSurface), sep='\t')

つく	で か
泣く	で
記憶	て


46. 動詞の格フレーム情報の抽出  
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．  
・項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）  
・述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる  
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．  
>始める  で      ここで  
>見る    は を   吾輩は ものを

In [19]:
for b in blockList[:5]:
    for m in b:
        if len(m.srcs) > 0:
            preMorphs = [b[int(s)].morphs for s in m.srcs]
            preMorphsFiltered = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in preMorphs]
            preSurface = [[p.surface for p in pm] for pm in preMorphsFiltered]
            preSurface = list(filter(lambda x: x != [], preSurface))
            preSurface = [p[0] for p in preSurface]
            postBase = [mo.base for mo in m.morphs]
            postPos = [mo.pos for mo in m.morphs]
            if len(preSurface) > 0 and '動詞' in postPos:
                preText = list(filter(lambda x: '助詞' in [p.pos for p in x], preMorphs))
                preText = [''.join([p.surface for p in pt]) for pt in preText]
                print(postBase[0], ' '.join(preSurface), ' '.join(preText), sep='\t')

つく	で か	　どこで 生れたか
泣く	で	所で
記憶	て	泣いて


47. 機能動詞構文のマイニング  
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．  
・「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする  
・述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる  
・述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる  
・述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）  
例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．  
>返事をする      と に は        及ばんさと 手紙に 主人は

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

・コーパス中で頻出する述語（サ変接続名詞+を+動詞）  
・コーパス中で頻出する述語と助詞パターン

In [20]:
# UNIXコマンドはパス
# サ変接続名詞とは

In [21]:
# with open('./Section_5/ans47.txt', mode='w') as f:
for b in blockList[:200]:
    for i, m in enumerate(b):
        if 'サ変接続' in [s.pos1 for s in m.morphs] and 'を' in [s.surface for s in m.morphs] and i + 1 < len(b) and b[i + 1].morphs[0].pos == '動詞':
            text = ''.join([s.surface for s in m.morphs]) + b[i + 1].morphs[0].base
            if len(m.srcs) > 0:
                preMorphs = [b[int(s)].morphs for s in m.srcs]
                preMorphsFiltered = [list(filter(lambda x: '助詞' in x.pos, pm)) for pm in preMorphs]
                preSurface = [[p.surface for p in pm] for pm in preMorphsFiltered]
                preSurface = list(filter(lambda x: x != [], preSurface))
                preSurface = [p[0] for p in preSurface]
                preText = list(filter(lambda x: '助詞' in [p.pos for p in x], preMorphs))
                preText = [''.join([p.surface for p in pt]) for pt in preText]
#                     if len(preSurface) > 0:
#                         f.writelines('\t'.join([text, ' '.join(preSurface), ' '.join(preText)]))
#                         f.write('\n')
                print('\t'.join([text, ' '.join(preSurface), ' '.join(preText)]))

返報をする	んで	偸んで
家族的生活をする	て	くして
話をする	で	我儘で
話をする	に	時に
失笑するのを禁じる	を て	有様を 見て
彩色を見る		
欠伸をする		


48. 名詞から根へのパスの抽出  
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ．  
ただし，構文木上のパスは以下の仕様を満たすものとする．  
・各文節は（表層形の）形態素列で表現する  
・パスの開始文節から終了文節に至るまで，各文節の表現を” -> “で連結する  
「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．  
>吾輩は -> 見た  
>ここで -> 始めて -> 人間という -> ものを -> 見た  
>人間という -> ものを -> 見た  
>ものを -> 見た

In [22]:
for b in blockList[:5]:
    for m in b:
        text = []
        if '名詞' in [s.pos for s in m.morphs] and int(m.dst) != -1:
            currentChunk = m
            text.append(''.join([m.surface for m in currentChunk.morphs]))
            nextChunk = b[int(currentChunk.dst)]
            while int(currentChunk.dst) != -1:
                text.append(''.join([m.surface for m in nextChunk.morphs]))
                currentChunk = nextChunk
                nextChunk = b[int(nextChunk.dst)]
            print(*text, sep=' -> ')

名前は -> 無い。
　どこで -> つかぬ。
何でも -> 所で -> 泣いて -> 記憶している。
所で -> 泣いて -> 記憶している。
ニャーニャー -> 記憶している。


49. 名詞間の係り受けパスの抽出  
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．  
ただし，名詞句ペアの文節番号がiとj（i<j）のとき，係り受けパスは以下の仕様を満たすものとする．  
・問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を” -> “で連結して表現する  
・文節iとjに含まれる名詞句はそれぞれ，XとYに置換する  
また，係り受けパスの形状は，以下の2通りが考えられる．  
・文節iから構文木の根に至る経路上に文節jが存在する場合: 文節iから文節jのパスを表示  
・上記以外で，文節iと文節jから構文木の根に至る経路上で共通の文節kで交わる場合: 文節iから文節kに至る直前のパスと文節jから文節kに至る直前までのパス，文節kの内容を” | “で連結して表示  
例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．  
>Xは | Yで -> 始めて -> 人間という -> ものを | 見た  
>Xは | Yという -> ものを | 見た  
>Xは | Yを | 見た  
>Xで -> 始めて -> Y  
>Xで -> 始めて -> 人間という -> Y  
>Xという -> Y

In [23]:
def convert(s):
    pl, nl = [], [c for c in s if '名詞' in [m.pos for m in c.morphs]]
    for i in range(len(nl) - 1):
        st1 = [''.join([m.surface if m.pos != '名詞' else 'X' for m in nl[i].morphs])]
        for e in nl[i + 1:]:
            dst, p = nl[i].dst, []
            st2 = [''.join([m.surface if m.pos != '名詞' else 'Y' for m in e.morphs])]
            while int(dst) != -1 and dst != s.index(e):
                p.append(s[int(dst)])
                dst = s[int(dst)].dst
            if len(p) < 1 or p[-1].dst != -1:
                mid = [''.join([m.surface for m in c.morphs if m.pos != '記号']) for c in p]
                pl.append(st1 + mid + ['Y'])
            else:
                mid, dst = [], e.dst
                while not s[int(dst)] in p:
                    mid.append(''.join([m.surface for m in s[int(dst)].morphs if m.pos != '記号']))
                    dst = s[int(dst)].dst
                ed = [''.join([m.surface for m in s[int(dst)].morphs if m.pos != '記号'])]
                pl.append([st1, st2 + mid, ed])
    return pl

In [24]:
for b in blockList[:5]:
    pl = (convert(b))
    for p in pl:
        if isinstance(p[0], str):
            print(' -> '.join(p))
        else:
            print(p[0][0], ' -> '.join(p[1]), p[2][0], sep=' | ')

Xは -> Y
　Xで -> つかぬ -> Y
Xでも -> 所で -> 泣いて -> 記憶している -> Y
Xでも -> 所で -> 泣いて -> 記憶している -> Y
Xでも -> 所で -> 泣いて -> 記憶している -> Y
Xでも -> 所で -> 泣いて -> 記憶している -> Y
Xで -> 泣いて -> 記憶している -> Y
Xで -> 泣いて -> 記憶している -> Y
Xで -> 泣いて -> 記憶している -> Y
X -> 記憶している -> Y
X -> 記憶している -> Y
Xだけは -> Y


In [ ]:
# あってる？